#GPT, BERT 입력값 만들기 
문장을 토큰화하고 결과값인 토큰들을 모델의 입력으로 만드는 과정 실습

In [2]:
#의존성 패키지 설치
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 816 kB/s 
     |████████████████████████████████| 582 kB 54.5 MB/s 
     |████████████████████████████████| 57 kB 6.6 MB/s 
     |████████████████████████████████| 2.8 MB 43.3 MB/s 
     |████████████████████████████████| 141 kB 57.9 MB/s 
     |████████████████████████████████| 419 kB 63.0 MB/s 
     |████████████████████████████████| 596 kB 71.2 MB/s 
     |████████████████████████████████| 880 kB 46.9 MB/s 
     |████████████████████████████████| 3.3 MB 49.9 MB/s 
     |████████████████████████████████| 101 kB 13.0 MB/s 
     |████████████████████████████████| 96 kB 1.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=41f114fad6478448a42415c6e9ce57d81519af73f2d6f9a77d4eac8a0f498dc8
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Su

#GPT 입력값 만들기 
tokenizer setting 코드에서 만든 바이트 기준 BPE 어휘집합과 바이그램 쌍의 병합 우선순위가 있어야 한다.  
(vocab.json, merge.txt)

In [5]:
#GPT 토크나이저 선언
from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/nlpbook/bbpe")
tokenizer_gpt.pad_token = "[PAD]"

file /content/drive/MyDrive/nlpbook/bbpe/config.json not found


In [7]:
#GPT 토크나이저로 토큰화하기
sentences = [
    "아 더빙.. 진짜 짜증나네요 목소리",
    "흠...포스터 보고 초딩영환줄... 오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]

In [8]:
#코드 실행 결과 확인 
tokenized_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'íı¬ìĬ¤íĦ°',
  'Ġë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĺģíĻĺ',
  'ì¤Ħ',
  '...',
  'Ġìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '..']]

In [10]:
#GPT 모델 입력 모델 만들기 
batch_inputs = tokenizer_gpt(
    sentences,
    padding = "max_length" , #문장의 최대 길이에 맞춰 패딩
    max_length = 12,  #문장의 토큰 기준 최대 길이
    truncation = True,  #문장 잘림 허용 옵션 

)

위 코드를 돌리면 두가지 input값이 나옵니다.  
* input_ids
* attention_mask  

In [14]:
#input_ids 확인
batch_inputs["input_ids"]

[[334, 2338, 263, 581, 4055, 464, 3808, 0, 0, 0, 0, 0],
 [3693, 336, 2876, 646, 2883, 5048, 806, 336, 4444, 875, 2960, 7292],
 [4957, 451, 3653, 263, 0, 0, 0, 0, 0, 0, 0, 0]]

**indexing** 
= 각 토큰을 인덱스로 변환하는 과정  

위 결과값을 보면 토큰 수가 총 12개 임을 볼 수 있다.  
(이유는 max_length에 12를 넣었기 때문)  
그보다 결과가 작게 나오면 PAD 토큰에 해당하는 0이 붙는다.  이때 PAD 토큰은 dummy token으로 길이를 맞춰주는 역할을 한다.  
문장 2의 경우에는 원래 토큰의 길이가 15이지만 인자값인 truncation = True 옵션 때문에 max_length인 12로 문장이 잘렸다. 


In [15]:
#attention_mask 확인
batch_inputs["attention_mask"]

[[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]

attention_mask는 일반 토큰이 자리한 곳과 패딩 토큰이 자리한 곳을 구분해 알려주는 장치이다. 

#BERT 입력값 만들기 
BERT 모델에서는 wordpiece 토크나이저를 사용하기 때문에 앞선 코드에서 만든 BERT용 워드피스 어휘집합이 있어햐 한다. 

In [16]:
#BERT 토크나이저 선언 
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained(
    "/content/drive/MyDrive/nlpbook/wordpiece",
    do_lower_case = False,
)

file /content/drive/MyDrive/nlpbook/wordpiece/config.json not found


In [19]:
#Wordpiece tokenizer 를 이용해서 토큰화를 진행 
sentences = [
    "아 더빙..진짜 짜증나네요 목소리",
    "흠...포스터보고 초딩영화인줄...오버연기조차 가볍지 않구나",
    "별루 였다..",
]
tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

In [20]:
#BERT 모델 입력 만들기 
batch_inputs = tokenizer_bert(
    sentences,
    padding = "max_length",
    max_length = 12,
    truncation = True,
)

위 코드의 결과값으로 3가지 입력값이 만들어지게 된다.
* input_ids
* attention_mask
* token_type_ids = segment에 해당 (모두 0) 

In [21]:
#input_ids 확인
batch_inputs["input_ids"]

[[2, 621, 2631, 16, 16, 1993, 3678, 1990, 3323, 3, 0, 0],
 [2, 997, 16, 16, 16, 2609, 2045, 2796, 1981, 2697, 16, 3],
 [2, 3274, 9507, 16, 16, 3, 0, 0, 0, 0, 0, 0]]

위 코드 실행 결과를 살펴보면 시작과 끝에 2와 3이 붙은 것을 확인할 수 있다.  
이는 각각 [CLS], [SEP] 이라는 토큰에 대응하는 인덱스로서 BERT 모델에서 이 2개의 토큰을 이용해서 문장의 시작과 끝을 나타내게 된다. 

In [22]:
#attention_mask 확인
batch_inputs["attention_mask"]

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]

In [23]:
#token_type_ids
batch_inputs["token_type_ids"]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

위 결과값은 세크먼트에 해당하는 것인데 세그먼트 정보는 BERT모델의 특징이다.  
BERT 모델은 문서나 문장 2개를 입력받는데 보통 token_type_ids로 판단한다.  
첫번째에 해당하면 token_type_ids가 0이고 두번째이면 1로 판단된다.  
그러나 예시 코드에서 문장을 하나씩 넣었으므로 모두 0으로 처리된 결과값을 확인할 수 있다. 